# SQL Basics using Python and SQLite library

We will be working with a dataset from the bike-sharing service Hubway (Hubway Bicycles Boston), which includes data on over 1.5 million trips made with the service saved in the hubway.db file. The dataset can be downloaded from [here](https://dataquest.io/blog/large_files/hubway.db). The database has two tables, trips and stations. To begin with, we'll just look at the trips table, which contains the following columns:

    id — A unique integer that serves as a reference for each trip
    duration — The duration of the trip, measured in seconds
    start_date — The date and time the trip began
    start_station — An integer that corresponds to the id column in the stations table for the station the trip started at
    end_date — The date and time the trip ended
    end_station — The 'id' of the station the trip ended at
    bike_number — Hubway's unique identifier for the bike used on the trip
    sub_type — The subscription type of the user. "Registered" for users with a membership, "Casual" for users without a membership
    zip_code — The zip code of the user (only available for registered members)
    birth_date — The birth year of the user (only available for registered members)
    gender — The gender of the user (only available for registered members)

We will be using a database system called SQLite3. SQLite comes as part of Python from version 2.5 onwards. So if you have Python installed, you will most probably have SQLite as part of the installation. Using Python to run our SQL code allows us to import the results into a Pandas dataframe to make it easier to display our results in an easy to read format. It also means we can perform further analysis and visualization on the data we pull from the database. 

## Our Analysis

We will try to answer the following questions in our analyses:

    1. What was the duration of the longest trip?
    2. How many trips were taken by 'registered' users?
    3. What was the average trip duration?
    4. Do registered or casual users take longer trips?
    5. Which bike was used for the most trips?
    6. What is the average duration of trips by users over the age of 30?


In [2]:
# Import the libraries
import sqlite3
import pandas as pd

In [3]:
# Connect to the database
db = sqlite3.connect('hubway.db')

In [5]:
# Use pandas pd.read_sql_query to read the sql query
def run_query(query):
    '''
    This function that takes our query (stored as a string) 
    as an input and shows the result as a formatted dataframe:
    '''
    return pd.read_sql_query(query, db)

## SELECT
It tells the database which columns you want to see. You can either specify columns by name (separated by commas) or use the wildcard * to return every column in the table. 

## LIMIT 
It simply tells the database how many rows you want it to return.

## ORDER BY
This command allows you to sort the database on a given column. To use it, you simply specify the name of the column you would like to sort on. By default, ORDER BY sorts in ascending order

In [10]:
query = 'SELECT * FROM trips LIMIT 5'
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981.0,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983.0,Female


In [11]:
query = 'SELECT duration, start_date FROM trips LIMIT 5'
run_query(query)

,duration,start_date
0,9,2011-07-28 10:12:00
1,220,2011-07-28 10:21:00
2,56,2011-07-28 10:33:00
3,64,2011-07-28 10:35:00
4,12,2011-07-28 10:37:00


In [12]:
# Question 1
query = '''
SELECT duration 
FROM trips
ORDER BY duration DESC
LIMIT 1;
'''

run_query(query)


,duration
0,9999


The longest trip lasted 9999 seconds or a little over 166 minutes. 

To make sure that the particularly long trips are not being cut by the database, let's try running the same query as before, but adjust the LIMIT to return the 10 highest durations and see if that's the case:

In [14]:
# Question 1
query = '''
SELECT duration 
FROM trips
ORDER BY duration DESC
LIMIT 10;
'''

run_query(query)

,duration
0,9999
1,9998
2,9998
3,9997
4,9996
5,9996
6,9995
7,9995
8,9994
9,9994


What we see here is that there aren't a whole bunch of trips at 9999, so it doesn't look like we're cutting off the top end of our durations, but it's still difficult to tell whether that's the real length of the trip or just the maximum allowed value. Hubway charges additional fees for rides over 30 minutes (somebody keeping a bike for 9999 seconds would have to pay an extra $25 in fees) so it's plausible that they decided 4 digits would be sufficient to track the majority of rides.

## WHERE
The WHERE command allows you to use a logical operator to specify which rows should be returned.

Let's write a query that uses WHERE to return every column in the trips table for each row with a duration longer than 9990 seconds:

In [15]:
query = '''
SELECT * 
FROM trips
WHERE duration > 9990
'''

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,4768,9994,2011-08-03 17:16:00,22,2011-08-03 20:03:00,24,B00002,Casual,,,
1,8448,9991,2011-08-06 13:02:00,52,2011-08-06 15:48:00,24,B00174,Casual,,,
2,11341,9998,2011-08-09 10:42:00,40,2011-08-09 13:29:00,42,B00513,Casual,,,
3,24455,9995,2011-08-20 12:20:00,52,2011-08-20 15:07:00,17,B00552,Casual,,,
4,55771,9994,2011-09-14 15:44:00,40,2011-09-14 18:30:00,40,B00139,Casual,,,
5,81191,9993,2011-10-03 11:30:00,22,2011-10-03 14:16:00,36,B00474,Casual,,,
6,89335,9997,2011-10-09 02:30:00,60,2011-10-09 05:17:00,45,B00047,Casual,,,
7,124500,9992,2011-11-09 09:08:00,22,2011-11-09 11:55:00,40,B00387,Casual,,,
8,133967,9996,2011-11-19 13:48:00,4,2011-11-19 16:35:00,58,B00238,Casual,,,
9,147451,9996,2012-03-23 14:48:00,35,2012-03-23 17:35:00,33,B00550,Casual,,,


Now let's write a query that uses WHERE to return every column in the trips table for each row with a duration longer than 9990 seconds and also had a sub_type of Registered:

In [16]:
query = '''
SELECT * 
FROM trips
WHERE (duration >= 9990) AND (sub_type = "Registered")
ORDER BY duration DESC;
'''

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,315737,9995,2012-07-03 18:28:00,12,2012-07-03 21:15:00,12,B00250,Registered,'02120,1964.0,Male


## COUNT
COUNT is a SQL aggregate function for counting the number of rows in a particular column.

In [19]:
# Question 2
query = '''
SELECT COUNT(*) AS "Total Trips by Registered Users"
FROM trips
WHERE sub_type = "Registered";
'''

run_query(query)

,Total Trips by Registered Users
0,1105192


## AVG
AVG is a SQL aggregate function that calculates the average of a selected group of values.

In [21]:
# Question 3
query = '''
SELECT AVG(duration) AS "Average Duration"
FROM trips;
'''

run_query(query)

,Average Duration
0,912.409682


## GROUP BY
GROUP BY allows us to separate data into groups, which can be aggregated independently of one another.

In [23]:
# Question 4
query = '''
SELECT sub_type, AVG(duration) AS "Average Duration"
FROM trips
GROUP BY sub_type;
'''

run_query(query)

,sub_type,Average Duration
0,Casual,1519.643897
1,Registered,657.026067


On average, registered users take trips that last around 11 minutes whereas casual users are spending almost 25 minutes per ride. Registered users are likely taking shorter, more frequent trips, possibly as part of their commute to work. Casual users, on the other hand, are spending around twice as long per trip. It's possible that casual users tend to come from demographics (tourists, for example) that are more inclined to take longer trips make sure they get around and see all the sights.

In [24]:
# Question 5
query = '''
SELECT bike_number as "Bike Number", COUNT(*) AS "Number of Trips"
FROM trips
GROUP BY bike_number
ORDER BY COUNT(*) DESC
LIMIT 1;
'''

run_query(query)

,Bike Number,Number of Trips
0,B00490,2120


As you can see from the output, bike B00490 took the most trips. 

In [31]:
# Question 6
query = '''
SELECT AVG(duration)
FROM trips
WHERE (sub_type = "Registered") AND ((2018 - birth_date) > 30);
'''

run_query(query)

,AVG(duration)
0,658.040633


So far we've only been looking at the trips table, but there's also a second table *stations* that contains information about every station in the Hubway network and includes an *id* column that is referenced by the *trips* table

In [32]:
query = '''
SELECT * 
FROM stations
LIMIT 5;
'''

run_query(query)

,id,station,municipality,lat,lng
0,3,Colleges of the Fenway,Boston,42.340021,-71.100812
1,4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
2,5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179
3,6,Cambridge St. at Joy St.,Boston,42.361284999999995,-71.06514
4,7,Fan Pier,Boston,42.353412,-71.044624


Description of the table:
    
    id — A unique identifier for each station (corresponds to the start_station and end_station columns in the trips table)
    station — The station name
    municipality — The municipality that the station is in (Boston, Brookline, Cambridge or Somerville)
    lat — The latitude of the station
    lng — The longitude of the station


We will to answer the following questions:

    6. Which stations are most frequently used for round trips?
    7. How many trips start and end in different municipalities?


Like before, we'll try to answer some questions in the data, starting with which station is the most frequent starting point?

## JOIN
We can use JOINS to answer the above questions as we need information from both tables to answer these questions. JOIN specifies which tables should be connected and ON specifies which columns in each table are related. 

In [34]:
query = '''
SELECT stations.station AS Station, COUNT(*) AS Count
FROM trips
JOIN stations 
ON trips.start_station = stations.id
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''

run_query(query)

,Station,Count
0,South Station - 700 Atlantic Ave.,56123
1,Boston Public Library - 700 Boylston St.,41994
2,Charles Circle - Charles St. at Cambridge St.,35984
3,Beacon St / Mass Ave,35275
4,MIT at Mass Ave / Amherst St,33644


In [35]:
# Question 7
query = '''
SELECT stations.station AS Station, COUNT(*) AS Count
FROM trips
JOIN stations 
ON trips.start_station = stations.id
WHERE trips.start_station = trips.end_station
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''

run_query(query)

,Station,Count
0,The Esplanade - Beacon St. at Arlington St.,3064
1,Charles Circle - Charles St. at Cambridge St.,2739
2,Boston Public Library - 700 Boylston St.,2548
3,Boylston St. at Arlington St.,2163
4,Beacon St / Mass Ave,2144


In [37]:
# Question 8
query = '''SELECT COUNT(trips.id) AS Count
FROM trips
JOIN stations AS start
ON trips.start_station = start.id 
JOIN stations AS end
ON trips.end_station = end.id
WHERE start.municipality <> end.municipality;
'''

run_query(query)

,Count
0,309748


This shows that about 300k out of 1.5 million trips (or 20%) ended in a different municipality than they started in - further evidence that people mostly use Hubway bicycles for relatively short journeys rather than longer trips between towns.

More questions to analyze:
    
    9. How many trips incurred additional fees (lasted longer than 30 minutes)?
    10. Which bike was used for the longest total time?
    11. Did registered or casual users take more round trips?
    12. Which municipality had the longest average duration?


This notebook is inspired from Dataquest's Data Science Blog [SQL Basics: Working with Databases](https://www.dataquest.io/blog/sql-basics/)